<a href="https://colab.research.google.com/github/thenazar9/ab-testing-statistical-analysis-python-tableau/blob/main/A_B_Testing_Statistical_Analysis_and_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A/B Testing Statistical Analysis and Visualization**

### **Project Overview**

This project implements a Python script to calculate the statistical significance of A/B test results across various user segments. By leveraging data from Google BigQuery, the script dynamically analyzes key conversion metrics such as `add_payment_info / session`, `add_shipping_info / session`, `begin_checkout / session`, and `new_accounts / session`. Unlike manual online calculators, this automated approach efficiently computes z-statistics and p-values for the overall test and also provides granular insights by segmenting results based on traffic channel, device type, and top-performing countries. The final output is a structured DataFrame containing all necessary information for subsequent data visualization, enabling a deeper understanding of how different user groups respond to the A/B test.

### **Importing Libraries and Setting Up Google BigQuery**

In [ ]:
# Installing the Google Cloud BigQuery package
!pip install --upgrade google-cloud-bigquery

In [ ]:
# Auth & BigQuery
from google.colab import auth
from google.cloud import bigquery

# Import libraries for analysis
import numpy as np
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

# Mounting Google Drive and copying files
from google.colab import drive
import shutil

In [ ]:
# Authenticate user for Google Cloud
auth.authenticate_user()

In [ ]:
# Initialize BigQuery client for the specified project
client = bigquery.Client(project="data-analytics-mate")

In [ ]:
"""
This SQL query combines data from multiple sources to calculate important metrics
about user activity, including sessions, events, orders, and new account registrations.

It performs the following steps:
1. Retrieves session data and links it to A/B test data (session_info).
2. Counts sessions that resulted in orders (session_with_orders).
3. Counts the number of specific events in each session (events).
4. Counts the total number of sessions (session).
5. Counts the number of new accounts created during each session (account).

The results from all these queries are combined using UNION ALL, providing a comprehensive view of sessions,
events, new account creation, and sessions with orders.
"""

query = """
WITH
  session_info AS(
  SELECT
    s.date,
    s.ga_session_id,
    sp.country,
    sp.device,
    sp.continent,
    sp.channel,
    ab.test,
    ab.test_group
  FROM
    `data-analytics-mate.DA.ab_test` ab
  JOIN
    `DA.session` s
  ON
    ab.ga_session_id = s.ga_session_id
  JOIN
    `DA.session_params` sp
  ON
    sp.ga_session_id = s.ga_session_id ),
  session_with_orders AS(
  SELECT
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    COUNT(DISTINCT o.ga_session_id) AS session_with_orders
  FROM
    `DA.order` o
  JOIN
    session_info
  ON
    o.ga_session_id = session_info.ga_session_id
  GROUP BY
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group ),
  events AS(
  SELECT
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    ep.event_name,
    COUNT(ep.ga_session_id) AS event_cnt
  FROM
    `DA.event_params` ep
  JOIN
    session_info
  ON
    ep.ga_session_id = session_info.ga_session_id
  GROUP BY
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    ep.event_name ),
  session AS(
  SELECT
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    COUNT(DISTINCT session_info.ga_session_id) AS session_cnt
  FROM
    session_info
  GROUP BY
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group ),
  account AS(
  SELECT
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    COUNT(DISTINCT acs.ga_session_id) AS new_accounts_cnt
  FROM
    `DA.account_session` acs
  JOIN
    session_info
  ON
    session_info.ga_session_id = acs.ga_session_id
  GROUP BY
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group )
SELECT
  session_with_orders.date,
  session_with_orders.country,
  session_with_orders.device,
  session_with_orders.continent,
  session_with_orders.channel,
  session_with_orders.test,
  session_with_orders.test_group,
  "session with orders" AS event_name,
  session_with_orders.session_with_orders AS value
FROM
  session_with_orders
UNION ALL
SELECT
  events.date,
  events.country,
  events.device,
  events.continent,
  events.channel,
  events.test,
  events.test_group,
  event_name,
  event_cnt AS value
FROM
  events
UNION ALL
SELECT
  session.date,
  session.country,
  session.device,
  session.continent,
  session.channel,
  session.test,
  session.test_group,
  "session" AS event_name,
  session_cnt AS value
FROM
  session
UNION ALL
SELECT
  account.date,
  account.country,
  account.device,
  account.continent,
  account.channel,
  account.test,
  account.test_group,
  "new account" AS event_name,
  new_accounts_cnt AS value
FROM
  account
"""

In [ ]:
# Run the query and get the result
query_job = client.query(query)
results = query_job.result()

In [ ]:
df = results.to_dataframe()

In [ ]:
df.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Lithuania,mobile,Europe,Organic Search,2,2,new account,1
1,2020-11-01,El Salvador,desktop,Americas,Social Search,2,1,new account,1
2,2020-11-01,Slovakia,mobile,Europe,Paid Search,2,2,new account,1
3,2020-11-01,Lithuania,desktop,Europe,Paid Search,2,2,new account,1
4,2020-11-02,North Macedonia,desktop,Europe,Direct,2,1,new account,1


In [ ]:
# Metrics list
metrics = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new account']

### **A/B Test Results Calculation by Segment (Total, Traffic Channel, Device Type, Countries)**

**Total A/B Test Results Calculation**

In [ ]:
def calculate_ab_test_results(df, metrics, alpha=0.05):
    # Initialize an empty list to store the results
    results = []

    # Loop through each unique test
    for test_number in df['test'].unique():
        test_df = df[df['test'] == test_number]

        # Loop through each metric
        for metric in metrics:
            # Separate data for group A (test group 1) and group B (test group 2)
            group_a = test_df[test_df['test_group'] == 1]
            group_b = test_df[test_df['test_group'] == 2]

            # Calculate the numerator and denominator for both groups for the given metric
            numerator_a = group_a[group_a['event_name'] == metric]['value'].sum()
            denominator_a = group_a[group_a['event_name'] == 'session']['value'].sum()
            numerator_b = group_b[group_b['event_name'] == metric]['value'].sum()
            denominator_b = group_b[group_b['event_name'] == 'session']['value'].sum()

            # Skip the metric if either denominator is zero
            if denominator_a == 0 or denominator_b == 0:
                continue

            # Calculate conversion rates for both groups
            conversion_rate_a = numerator_a / denominator_a
            conversion_rate_b = numerator_b / denominator_b

            # Calculate the percentage change between the two groups
            metric_change = (conversion_rate_b / conversion_rate_a) - 1

            # Perform z-test to check statistical significance
            count = np.array([numerator_b, numerator_a])
            nobs = np.array([denominator_b, denominator_a])
            z_stat, p_value = proportions_ztest(count, nobs)

            # Check if the p-value is less than the significance level (alpha)
            significant = p_value < alpha

            # Append the results to the list
            results.append({
                'test_number': test_number,
                'metric': metric,
                'numerator_converse_B': numerator_b,
                'denominator_converse_B': denominator_b,
                'conversion_rate_B': conversion_rate_b,
                'numerator_converse_A': numerator_a,
                'denominator_converse_A': denominator_a,
                'conversion_rate_A': conversion_rate_a,
                'metric_change': metric_change,
                'z_stat': z_stat,
                'p_value': p_value,
                'significant': significant
            })

    # Return the results as a DataFrame
    return pd.DataFrame(results)


In [ ]:
# Calculates A/B test results (conversion rates, metric change, z-test) for given metrics.
calculate_ab_test_results(df, metrics)

,test_number,metric,numerator_converse_B,denominator_converse_B,conversion_rate_B,numerator_converse_A,denominator_converse_A,conversion_rate_A,metric_change,z_stat,p_value,significant
0,2,add_payment_info,2409,50244,0.047946,2344,50637,0.046290,0.035769,1.240994,0.214608,False
1,2,add_shipping_info,3510,50244,0.069859,3480,50637,0.068724,0.016510,0.709557,0.477979,False
2,2,begin_checkout,4313,50244,0.085841,4262,50637,0.084168,0.019882,0.952898,0.340642,False
3,2,new account,4184,50244,0.083274,4165,50637,0.082252,0.012419,0.588793,0.556000,False
4,1,add_payment_info,2229,45193,0.049322,1988,45362,0.043825,0.125420,3.924884,0.000087,True
5,1,add_shipping_info,3221,45193,0.071272,3034,45362,0.066884,0.065605,2.603571,0.009226,True
6,1,begin_checkout,4021,45193,0.088974,3784,45362,0.083418,0.066606,2.978783,0.002894,True
7,1,new account,3681,45193,0.081451,3823,45362,0.084278,-0.033543,-1.542883,0.122859,False
8,4,add_payment_info,3601,105141,0.034249,3731,105079,0.035507,-0.035412,-1.571106,0.116158,False
9,4,add_shipping_info,4956,105141,0.047137,5128,105079,0.048801,-0.034111,-1.785795,0.074132,False


**A/B Test Results by Traffic Channel**

In [ ]:
def calculate_ab_test_results_by_traffic_channel(df, metrics, alpha=0.05):
    # Initialize an empty list to store the results
    results = []

    # Loop through each unique traffic channel
    for traffic_channel in df["channel"].unique():
        channel_df = df[df["channel"] == traffic_channel]

        # Loop through each unique test within the traffic channel
        for test_number in df['test'].unique():
            test_df = channel_df[channel_df['test'] == test_number]

            # Loop through each metric
            for metric in metrics:
                # Separate data for group A (test group 1) and group B (test group 2)
                group_a = test_df[test_df['test_group'] == 1]
                group_b = test_df[test_df['test_group'] == 2]

                # Calculate the numerator and denominator for both groups for the given metric
                numerator_a = group_a[group_a['event_name'] == metric]['value'].sum()
                denominator_a = group_a[group_a['event_name'] == 'session']['value'].sum()
                numerator_b = group_b[group_b['event_name'] == metric]['value'].sum()
                denominator_b = group_b[group_b['event_name'] == 'session']['value'].sum()

                # Skip the metric if either denominator is zero
                if denominator_a == 0 or denominator_b == 0:
                    continue

                # Calculate conversion rates for both groups
                conversion_rate_a = numerator_a / denominator_a
                conversion_rate_b = numerator_b / denominator_b

                # Calculate the percentage change between the two groups
                metric_change = (conversion_rate_b / conversion_rate_a) - 1

                # Perform z-test to check statistical significance
                count = np.array([numerator_b, numerator_a])
                nobs = np.array([denominator_b, denominator_a])
                z_stat, p_value = proportions_ztest(count, nobs)

                # Check if the p-value is less than the significance level (alpha)
                significant = p_value < alpha

                # Append the results to the list
                results.append({
                    'traffic_channel': traffic_channel,
                    'test_number': test_number,
                    'metric': metric,
                    'numerator_converse_B': numerator_b,
                    'denominator_converse_B': denominator_b,
                    'conversion_rate_B': conversion_rate_b,
                    'numerator_converse_A': numerator_a,
                    'denominator_converse_A': denominator_a,
                    'conversion_rate_A': conversion_rate_a,
                    'metric_change': metric_change,
                    'z_stat': z_stat,
                    'p_value': p_value,
                    'significant': significant
                })

    # Return the results as a DataFrame
    return pd.DataFrame(results)

In [ ]:
# Calculates A/B test results (conversion rates, metric change, z-test) traffic channel for given metrics.
calculate_ab_test_results_by_traffic_channel(df, metrics, alpha=0.05)

,traffic_channel,test_number,metric,numerator_converse_B,denominator_converse_B,conversion_rate_B,numerator_converse_A,denominator_converse_A,conversion_rate_A,metric_change,z_stat,p_value,significant
0,Organic Search,2,add_payment_info,597,17428,0.034255,698,17466,0.039963,-0.142834,-2.820279,0.004798,True
1,Organic Search,2,add_shipping_info,1040,17428,0.059674,1130,17466,0.064697,-0.077639,-1.942671,0.052056,False
2,Organic Search,2,begin_checkout,1251,17428,0.071781,1385,17466,0.079297,-0.094781,-2.656343,0.007899,True
3,Organic Search,2,new account,1500,17428,0.086068,1422,17466,0.081415,0.057152,1.568956,0.116658,False
4,Organic Search,1,add_payment_info,514,15631,0.032883,640,15675,0.040829,-0.194614,-3.730758,0.000191,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,Undefined,4,new account,510,5862,0.087001,484,5716,0.084675,0.027475,0.446742,0.655061,False
76,Undefined,3,add_payment_info,542,4534,0.119541,492,4636,0.106126,0.126409,2.030631,0.042292,True
77,Undefined,3,add_shipping_info,655,4534,0.144464,587,4636,0.126618,0.140946,2.496906,0.012528,True
78,Undefined,3,begin_checkout,1175,4534,0.259153,1066,4636,0.229940,0.127048,3.254797,0.001135,True


**A/B Test Results by Device Type**

In [ ]:
def calculate_ab_test_results_by_device(df, metrics, alpha=0.05):
    # Initialize an empty list to store the results
    results = []

    # Loop through each unique device
    for device in df["device"].unique():
        device_df = df[df["device"] == device]

        # Loop through each unique test within the device
        for test_number in df["test"].unique():
            test_df = device_df[device_df["test"] == test_number]

            # Loop through each metric
            for metric in metrics:
                # Separate data for group A (test group 1) and group B (test group 2)
                group_a = test_df[test_df['test_group'] == 1]
                group_b = test_df[test_df['test_group'] == 2]

                # Calculate the numerator and denominator for both groups for the given metric
                numerator_a = group_a[group_a["event_name"] == metric]["value"].sum()
                denominator_a = group_a[group_a["event_name"] == "session"]["value"].sum()
                numerator_b = group_b[group_b["event_name"] == metric]["value"].sum()
                denominator_b = group_b[group_b["event_name"] == "session"]["value"].sum()

                # Skip the metric if either denominator is zero
                if denominator_a == 0 or denominator_b == 0:
                    continue

                # Calculate conversion rates for both groups
                conversion_rate_a = numerator_a / denominator_a
                conversion_rate_b = numerator_b / denominator_b

                # Calculate the percentage change between the two groups
                metric_change = (conversion_rate_b / conversion_rate_a) - 1

                # Perform z-test to check statistical significance
                count = np.array([numerator_b, numerator_a])
                nobs = np.array([denominator_b, denominator_a])
                z_stat, p_value = proportions_ztest(count, nobs)

                # Check if the p-value is less than the significance level (alpha)
                significant = p_value < alpha

                # Append the results to the list
                results.append({
                    'device_type': device,
                    'test_number': test_number,
                    'metric': metric,
                    'numerator_converse_B': numerator_b,
                    'denominator_converse_B': denominator_b,
                    'conversion_rate_B': conversion_rate_b,
                    'numerator_converse_A': numerator_a,
                    'denominator_converse_A': denominator_a,
                    'conversion_rate_A': conversion_rate_a,
                    'metric_change': metric_change,
                    'z_stat': z_stat,
                    'p_value': p_value,
                    'significant': significant
                })

    # Return the results as a DataFrame
    return pd.DataFrame(results)

In [ ]:
# Calculates A/B test results (conversion rates, metric change, z-test) by device type for given metrics.
calculate_ab_test_results_by_device(df, metrics, alpha=0.005)

,device_type,test_number,metric,numerator_converse_B,denominator_converse_B,conversion_rate_B,numerator_converse_A,denominator_converse_A,conversion_rate_A,metric_change,z_stat,p_value,significant
0,mobile,2,add_payment_info,961,19756,0.048643,978,20017,0.048858,-0.004401,-0.099562,0.920692,False
1,mobile,2,add_shipping_info,1356,19756,0.068637,1417,20017,0.070790,-0.030406,-0.842754,0.399366,False
2,mobile,2,begin_checkout,1656,19756,0.083823,1766,20017,0.088225,-0.049899,-1.565423,0.117484,False
3,mobile,2,new account,1701,19756,0.086100,1623,20017,0.081081,0.061905,1.808493,0.070530,False
4,mobile,1,add_payment_info,942,17767,0.053020,810,17896,0.045262,0.171407,3.389330,0.000701,True
5,mobile,1,add_shipping_info,1256,17767,0.070693,1257,17896,0.070239,0.006459,0.167387,0.867065,False
6,mobile,1,begin_checkout,1561,17767,0.087860,1593,17896,0.089014,-0.012973,-0.384029,0.700957,False
7,mobile,1,new account,1441,17767,0.081105,1530,17896,0.085494,-0.051332,-1.499486,0.133748,False
8,mobile,4,add_payment_info,1501,41345,0.036304,1413,41216,0.034283,0.058964,1.573870,0.115517,False
9,mobile,4,add_shipping_info,2023,41345,0.048930,1961,41216,0.047579,0.028398,0.905776,0.365054,False


**A/B Test Results for Top 5 Countries**

In [ ]:
def calculate_ab_test_results_top_5_countries(df, metrics, alpha=0.05):
    # Initialize an empty list to store the results
    results = []

    # Filter the top 5 countries based on the highest session values
    df_top_5_countries = df[df["country"].isin(df[df["event_name"] == "session"].groupby("country")["value"].sum().nlargest(5).index)]

    # Loop through each unique country in the top 5 countries
    for country in df_top_5_countries["country"].unique():
        country_df = df_top_5_countries[df_top_5_countries["country"] == country]

        # Loop through each unique test within the country
        for test_number in country_df["test"].unique():
            test_df = country_df[country_df["test"] == test_number]

            # Loop through each metric
            for metric in metrics:
                # Separate data for group A (test group 1) and group B (test group 2)
                group_a = test_df[test_df['test_group'] == 1]
                group_b = test_df[test_df['test_group'] == 2]

                # Calculate the numerator and denominator for both groups for the given metric
                numerator_a = group_a[group_a["event_name"] == metric]["value"].sum()
                denominator_a = group_a[group_a["event_name"] == "session"]["value"].sum()
                numerator_b = group_b[group_b["event_name"] == metric]["value"].sum()
                denominator_b = group_b[group_b["event_name"] == "session"]["value"].sum()

                # Skip the metric if either denominator is zero
                if denominator_a == 0 or denominator_b == 0:
                    continue

                # Calculate conversion rates for both groups
                conversion_rate_a = numerator_a / denominator_a
                conversion_rate_b = numerator_b / denominator_b

                # Calculate the percentage change between the two groups
                metric_change = (conversion_rate_b / conversion_rate_a) - 1

                # Perform z-test to check statistical significance
                count = np.array([numerator_b, numerator_a])
                nobs = np.array([denominator_b, denominator_a])
                z_stat, p_value = proportions_ztest(count, nobs)

                # Check if the p-value is less than the significance level (alpha)
                significant = p_value < alpha

                # Append the results to the list
                results.append({
                    'country': country,
                    'test_number': test_number,
                    'metric': metric,
                    'numerator_converse_B': numerator_b,
                    'denominator_converse_B': denominator_b,
                    'conversion_rate_B': conversion_rate_b,
                    'numerator_converse_A': numerator_a,
                    'denominator_converse_A': denominator_a,
                    'conversion_rate_A': conversion_rate_a,
                    'metric_change': metric_change,
                    'z_stat': z_stat,
                    'p_value': p_value,
                    'significant': significant
                })

    # Return the results as a DataFrame
    return pd.DataFrame(results)

In [ ]:
# Calculates A/B test results (conversion rates, metric change, z-test) for the top 5 countries by session count for given metrics.
calculate_ab_test_results_top_5_countries(df, metrics, alpha=0.05)

,country,test_number,metric,numerator_converse_B,denominator_converse_B,conversion_rate_B,numerator_converse_A,denominator_converse_A,conversion_rate_A,metric_change,z_stat,p_value,significant
0,Canada,2,add_payment_info,146,3663,0.039858,211,3756,0.056177,-0.290489,-3.283543,1.025110e-03,True
1,Canada,2,add_shipping_info,221,3663,0.060333,308,3756,0.082002,-0.264250,-3.626242,2.875761e-04,True
2,Canada,2,begin_checkout,269,3663,0.073437,369,3756,0.098243,-0.252494,-3.810210,1.388490e-04,True
3,Canada,2,new account,291,3663,0.079443,321,3756,0.085463,-0.070442,-0.942345,3.460162e-01,False
4,Canada,1,add_payment_info,194,3307,0.058663,136,3368,0.040380,0.452783,3.445187,5.706644e-04,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,United States,4,new account,3845,46030,0.083532,3983,46178,0.086253,-0.031543,-1.482041,1.383293e-01,False
76,United States,3,add_payment_info,1602,30833,0.051957,1589,30734,0.051702,0.004944,0.143056,8.862460e-01,False
77,United States,3,add_shipping_info,2190,30833,0.071028,2380,30734,0.077439,-0.082786,-3.034062,2.412848e-03,True
78,United States,3,begin_checkout,3896,30833,0.126358,4322,30734,0.140626,-0.101460,-5.204813,1.941925e-07,True


### **Saving Results to Google Drive**

In [ ]:
# Calculate results for each segment
results_total = calculate_ab_test_results(df.copy(), metrics)
results_total['segment'] = 'total'

results_channel = calculate_ab_test_results_by_traffic_channel(df.copy(), metrics)
results_channel['segment'] = 'channel'

results_device = calculate_ab_test_results_by_device(df.copy(), metrics)
results_device['segment'] = 'device'

results_countries = calculate_ab_test_results_top_5_countries(df.copy(), metrics)
results_countries['segment'] = 'country'

# Concatenate all results into a single DataFrame
all_results = pd.concat([results_total, results_channel, results_device, results_countries])

all_results.to_csv('ab_test_results_all_segments.csv', index=False)

print(all_results.head().to_string())

   test_number             metric  numerator_converse_B  denominator_converse_B  conversion_rate_B  numerator_converse_A  denominator_converse_A  conversion_rate_A  metric_change    z_stat   p_value  significant segment traffic_channel device_type country
0            2   add_payment_info                  2409                   50244           0.047946                  2344                   50637           0.046290       0.035769  1.240994  0.214608        False   total             NaN         NaN     NaN
1            2  add_shipping_info                  3510                   50244           0.069859                  3480                   50637           0.068724       0.016510  0.709557  0.477979        False   total             NaN         NaN     NaN
2            2     begin_checkout                  4313                   50244           0.085841                  4262                   50637           0.084168       0.019882  0.952898  0.340642        False   total             

In [ ]:
# Mounting Google Drive
drive.mount('/content/drive')

# The path to the file on Google Drive
file_path_on_drive = '/content/drive/MyDrive/ab_test/ab_test_results_all_segments.csv'

# Copy the file from the local Colab environment to Google Drive
shutil.copy('ab_test_results_all_segments.csv', file_path_on_drive)

print(f"File 'ab_test_results_all_segments.csv' has been successfully copied to Google Drive at: {file_path_on_drive}")

Mounted at /content/drive
File 'ab_test_results_all_segments.csv' has been successfully copied to Google Drive at: /content/drive/MyDrive/ab_test/ab_test_results_all_segments.csv


### **File and Dashboard Links**

[**Results of A/B Tests - CSV**](https://drive.google.com/file/d/1omW2hqokNny8LdvefKFbgCErA7_zzKUu/view?usp=sharing)

[**Tableau Dashboard**](https://public.tableau.com/views/ABTestAnalysis_17459432485280/ABtest?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link)